### L'objectif de ce script est de charger les données en ligne de OpenFoodFacts pour les stocker dans un fichier décompressé Mongo DB Dump

## Panneau de configuration

In [1]:
projectPath = "/home/carolus/Documents/school/green_ia/data/"
fileNbr = '00' # numéro d'identification du csv à générer 
datasetOpenFoodFacts = "https://static.openfoodfacts.org/data/openfoodfacts-mongodbdump.gz"

In [2]:
import os
import requests
import gzip
import shutil
import pandas as pd
from requests.exceptions import RequestException

In [3]:
# nom fichier téléchargé
fileName = fileNbr + "_OpenFoodFacts.gz"
filePath = os.path.join(projectPath, fileName)

In [4]:
# fonction pour reprendre le téléchargement
def download_file(url, file_path):
    while True:
        try:
            # vérifier si le fichier existe déjà et obtenir sa taille
            file_size = 0
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path)

            headers = {"Range": f"bytes={file_size}-"}
            response = requests.get(url, headers=headers, stream=True)

            if response.status_code in [200, 206]:
                mode = 'ab' if file_size else 'wb'
                with open(file_path, mode) as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            file.write(chunk)
                print(f"fichier téléchargé et sauvegardé ici: {file_path}")
                break  # sortir de la boucle une fois le téléchargement terminé

            else:
                print(f"erreur: {response.status_code}")
                break  # sortir de la boucle en cas d'erreur de statut

        except RequestException as e:
            print(f"erreur, reprise du téléchargement : {e}")

download_file(datasetOpenFoodFacts, filePath)

erreur, reprise du téléchargement : ('Connection broken: IncompleteRead(1083829911 bytes read, 8300863313 more expected)', IncompleteRead(1083829911 bytes read, 8300863313 more expected))
erreur, reprise du téléchargement : ('Connection broken: IncompleteRead(1083152646 bytes read, 7217714402 more expected)', IncompleteRead(1083152646 bytes read, 7217714402 more expected))
erreur, reprise du téléchargement : ('Connection broken: IncompleteRead(1082021974 bytes read, 6135698834 more expected)', IncompleteRead(1082021974 bytes read, 6135698834 more expected))
erreur, reprise du téléchargement : ('Connection broken: IncompleteRead(1081983424 bytes read, 5053721640 more expected)', IncompleteRead(1081983424 bytes read, 5053721640 more expected))
fichier téléchargé et sauvegardé ici: /home/carolus/Documents/school/green_ia/data/00_OpenFoodFacts.gz


In [5]:
# décompresser le fichier gz
def decompress_gz(filePath, projectPath):
    
    with gzip.open(filePath, 'rb') as f_in:
        with open(projectPath, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

outputPath = projectPath + fileNbr + "_OpenFoodFacts"
decompress_gz(filePath, outputPath)